In [1]:
import os
import fitz  # PyMuPDF
import openai
import pytesseract
import tempfile
import base64
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from tqdm import tqdm
from pdf2image import convert_from_path
from PIL import Image
from openai import OpenAI

# ========== 1. 初始化 OpenAI ==========
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ========== 2. 提取文本：前后20页 ==========
def extract_front_back_text(pdf_path, front_n=10, back_n=10):
    try:
        doc = fitz.open(pdf_path)
        texts = [doc[i].get_text() for i in range(min(front_n, len(doc)))]
        texts += [doc[i].get_text() for i in range(max(0, len(doc) - back_n), len(doc))]
        doc.close()
        full_text = "\n".join(texts)
        if len(full_text.strip()) < 50:
            raise ValueError("Too little text")
        return full_text
    except Exception as e:
        print(f"⚠️ PyMuPDF failed on {pdf_path.name}, switching to OCR...")
        return extract_text_with_ocr(pdf_path)

# ========== 3. OCR 补救 ==========
def extract_text_with_ocr(pdf_path, dpi=300):
    with tempfile.TemporaryDirectory() as path:
        images = convert_from_path(pdf_path, dpi=dpi, output_folder=path)
        text_parts = [pytesseract.image_to_string(img) for img in images[:3] + images[-3:]]
        return "\n".join(text_parts)

# ========== 4. 构造 Prompt ==========
def build_report_year_prompt(text):
    return f"""
You are an expert assistant helping to extract reporting years from corporate reports.

Please read the following text carefully, and complete the following tasks:

1. Identify any phrase that refers to the **reporting period**, **fiscal year**, or **financial year**.
   Examples:
   - "for the year ended 31 December 2021"
   - "reporting period: April 2021 – March 2022"
   - "2011-14" or "FY2017"

2. Normalize the expression into a date range format:
   - Always use “to” instead of “–” or “-” (e.g. "April 2021 to March 2022")
   - If only year is provided (e.g. "2011-14"), convert to: "2011 to 2014"
   - If full date is provided, remove ordinal suffixes like “1st”, “2nd”, “31st”, etc.
   - Convert to long form like: "1 January 2021 to 31 December 2021"

3. If a valid expression is found, return a JSON with:
{{
  "normalized_report_year": "...",
  "original_expression": "...",
  "source": "e.g. Page 1, main text"
}}

4. If nothing is found, return:
{{
  "normalized_report_year": null,
  "original_expression": null,
  "source": "NOT FOUND"
}}

Only output the JSON object. Do not add explanations.

Text:
{text}
"""

# ========== 5. Vision 模型辅助 ==========
def encode_image_to_base64(pil_image):
    with tempfile.NamedTemporaryFile(suffix=".png") as f:
        pil_image.save(f.name, format="PNG")
        with open(f.name, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

def extract_year_from_vision(pdf_path, client, page_limit=3):
    try:
        images = convert_from_path(pdf_path, dpi=200)
        for i, img in enumerate(images[:page_limit]):
            b64 = encode_image_to_base64(img)
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "user", "content": [
                        {"type": "text", "text": f"Please extract the fiscal year or reporting period from page {i+1}. Please return in this JSON format:\n"
                                                 "{\n  \"normalized_report_year\": \"...\",\n  \"original_expression\": \"...\",\n  \"source\": \"Page {i+1}, image-based\"\n}"},
                        {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{b64}"}}
                    ]}
                ],
                max_tokens=300
            )
            content = response.choices[0].message.content
            parsed = json.loads(content.strip()) if content.strip().startswith("{") else eval(content.strip())
            if parsed.get("normalized_report_year"):
                return parsed
    except Exception as e:
        return {
            "normalized_report_year": None,
            "original_expression": None,
            "source": f"Vision ERROR: {e}"
        }
    return {
        "normalized_report_year": None,
        "original_expression": None,
        "source": "Vision NOT FOUND"
    }

# ========== 6. 主函数：先文本，再 Vision ==========
def extract_report_year(pdf_path, client):
    try:
        text = extract_front_back_text(pdf_path)
        prompt = build_report_year_prompt(text)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=500
        )
        content = response.choices[0].message.content
        parsed = json.loads(content.strip()) if content.strip().startswith("{") else eval(content.strip())
        if parsed.get("normalized_report_year"):
            return parsed
    except Exception as e:
        print(f"⚠️ GPT-4.1-mini failed on {Path(pdf_path).name}, fallback to Vision...")
    return extract_year_from_vision(pdf_path, client)


from joblib import Parallel, delayed

# 多线程处理函数
def process_pdf(pdf_path):
    try:
        out = extract_report_year(str(pdf_path), client)
        out["filename"] = pdf_path.name
    except Exception as e:
        out = {
            "filename": pdf_path.name,
            "normalized_report_year": None,
            "original_expression": None,
            "source": f"ERROR: {e}"
        }
    return out

def batch_extract_years_multithread(pdf_dir, output_csv="results/extracted_report_years_formal1.csv", n_jobs=2):
    pdf_dir = Path(pdf_dir)
    pdf_files = sorted(pdf_dir.glob("*.pdf"))
    os.makedirs(Path(output_csv).parent, exist_ok=True)

    # ✅ 并行处理
    results = Parallel(n_jobs=n_jobs, prefer="threads")(
        delayed(process_pdf)(pdf) for pdf in tqdm(pdf_files, desc="📄 Processing PDFs")
    )

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Extraction complete! Results saved to: {output_csv}")

# ========== 8. 运行 ==========
if __name__ == "__main__":
    batch_extract_years_multithread("pdf_folder")  

📄 Processing PDFs:   1%|          | 13/1277 [00:14<19:15,  1.09it/s]

⚠️ GPT-4.1-mini failed on APA_Corp_US_APA-SustainabilityReport-2022_puby5ax5.pdf, fallback to Vision...


📄 Processing PDFs:   2%|▏         | 21/1277 [00:30<34:04,  1.63s/it]

⚠️ GPT-4.1-mini failed on Ab_Inbev_Si_Limited_AB20InBev_202220ESG20Report_FINAL_ncbtav56.pdf, fallback to Vision...


📄 Processing PDFs:   3%|▎         | 32/1277 [00:58<34:42,  1.67s/it]  

...

⚠️ GPT-4.1-mini failed on Unknown_about-us-sustainability-report-and-policy-sustainability-report-2015-en_l4r0j0ra.pdf, fallback to Vision...
⚠️ GPT-4.1-mini failed on Unknown_acs04-csr-esp_2wgz8riy.pdf, fallback to Vision...


📄 Processing PDFs: 100%|█████████▉| 1274/1277 [49:52<00:13,  4.45s/it]

⚠️ GPT-4.1-mini failed on Unknown_adbi-managing-transition-low-carbon-economy_087is5zy.pdf, fallback to Vision...
⚠️ GPT-4.1-mini failed on Unknown_adp07-sus-fr_95qx6prh.pdf, fallback to Vision...


📄 Processing PDFs: 100%|██████████| 1277/1277 [49:54<00:00,  2.34s/it]



✅ Extraction complete! Results saved to: results/extracted_report_years_formal1.csv


In [3]:
import pandas as pd

# 读取文件
anno_df = pd.read_excel("check/rfyear_annotation.xlsx")
result_df = pd.read_csv("output/report_years_extracted_new.csv")

# 清洗文件名列
anno_df["pdf_name_clean"] = anno_df["pdf_name"].str.strip().str.lower()
result_df["filename_clean"] = result_df["filename"].str.strip().str.lower()

# 合并两个表
merged_df = pd.merge(
    anno_df,
    result_df,
    left_on="pdf_name_clean",
    right_on="filename_clean",
    how="left"
)

# 年份对比函数
def match_year(human, gpt):
    if pd.isna(human) or pd.isna(gpt):
        return "missing"
    return "match" if str(human).strip().lower() == str(gpt).strip().lower() else "mismatch"

# 添加匹配结果
merged_df["match_result"] = merged_df.apply(
    lambda row: match_year(row["chosen_rfyear"], row["report_year_vision"]),
    axis=1
)

# 仅保留有用字段
final_df = merged_df[[
    "pdf_name", "chosen_rfyear", "report_year_vision", "match_result"
]]

# 保存结果
final_df.to_csv("rfyear_comparison_results_minimal.csv", index=False)

# 输出统计
print("✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv")
print("📊 匹配情况统计：")
print(final_df["match_result"].value_counts())

✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv
📊 匹配情况统计：
match_result
missing     386
mismatch    215
Name: count, dtype: int64


In [4]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. 读取两个文件
gpt_df = pd.read_csv("results/extracted_report_years_formal.csv")  # 提取结果
label_df = pd.read_excel("check/rfyear_annotation.xlsx")         # 人工标注文件

# 2. 重命名列以便对齐合并
gpt_df.rename(columns={"filename": "pdf_name", "report_year": "report_year_pred"}, inplace=True)

# 3. 合并两个表（inner join，只保留两个都有的 pdf）
merged = pd.merge(label_df, gpt_df, on="pdf_name", how="inner")

# 4. 标准化字符串格式（去除空格大小写等）
merged["report_year_pred"] = merged["report_year_pred"].astype(str).str.strip().str.lower()
merged["chosen_rfyear"] = merged["chosen_rfyear"].astype(str).str.strip().str.lower()

# 5. 定义匹配逻辑（完全匹配即可）
merged["match"] = merged["report_year_pred"] == merged["chosen_rfyear"]

# 6. 统计指标
total = len(merged)
correct = merged["match"].sum()
accuracy = correct / total

print(f"✅ 合并样本数：{total}")
print(f"✅ 匹配正确数：{correct}")
print(f"✅ 准确率（accuracy）：{accuracy:.2%}")

# 7. 可选：保存对比结果
# merged[["pdf_name", "chosen_rfyear", "report_year_pred", "match"]].to_csv("eval/year_comparison.csv", index=False)

✅ 合并样本数：224
✅ 匹配正确数：5
✅ 准确率（accuracy）：2.23%


In [6]:
import pandas as pd
from sklearn.metrics import classification_report

# 1. 读取两个文件
gpt_df = pd.read_csv("results/extracted_report_years_formal.csv")
human_df = pd.read_excel("check/rfyear_annotation.xlsx")  # 修改为你的路径

# 2. 对齐列名并合并（按 PDF 文件名匹配）
gpt_df["pdf_name"] = gpt_df["filename"].str.replace(".pdf", "", regex=False)
merged_df = pd.merge(human_df, gpt_df, on="pdf_name", how="inner")

# 3. 标准化年份字符串（两侧空格、小写等）
merged_df["chosen_rfyear_std"] = merged_df["chosen_rfyear"].astype(str).str.strip().str.lower()
merged_df["report_year_std"] = merged_df["report_year"].astype(str).str.strip().str.lower()

# 4. 比对是否完全一致
merged_df["is_match"] = merged_df["chosen_rfyear_std"] == merged_df["report_year_std"]

# 5. 评估准确率
total = len(merged_df)
correct = merged_df["is_match"].sum()
accuracy = correct / total

# 6. 输出评估结果
print(f"✅ Total matched records: {total}")
print(f"✅ Correct matches: {correct}")
print(f"✅ Accuracy: {accuracy:.2%}")

# 7. 可视化错误案例
errors = merged_df[~merged_df["is_match"]][["pdf_name", "chosen_rfyear", "report_year", "source"]]
errors.to_csv("eval/year_mismatch_cases.csv", index=False)
print("🚨 Mismatched cases saved to eval/year_mismatch_cases.csv")

✅ Total matched records: 0
✅ Correct matches: 0
✅ Accuracy: nan%
🚨 Mismatched cases saved to eval/year_mismatch_cases.csv


/var/folders/9z/mm8bj8yn2yj6rk7vdlhpt07h0000gn/T/ipykernel_83488/234731121.py:22: RuntimeWarning: invalid value encountered in scalar divide
  accuracy = correct / total
